In [1]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from azure.storage.blob import BlobServiceClient

subscription_id = '96599b9b-0d9b-4577-9163-418d2c6cd411'
resource_group = 'rg-datasci-ml-dev-001'
workspace_name = 'ml-moss-dev-001'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset_hourly_compare_cmic = Dataset.get_by_name(workspace, name='Hourly_Employee_Time_prod')
cmic_pr_df_dev = dataset_hourly_compare_cmic.to_pandas_dataframe()

dataset_hourly_compare_wd = Dataset.get_by_name(workspace, name='WD_Hourly_Employee_Time_prod')
wd_pr_df_dev = dataset_hourly_compare_wd.to_pandas_dataframe()

tcp_dataset = Dataset.get_by_name(workspace, name='WorkSegments_Timesheet_prod')
tcp_df_dev = tcp_dataset.to_pandas_dataframe()

jcjob_dataset = Dataset.get_by_name(workspace, name='JCJOB_TABLE_TN_prod')
jcjob_df = jcjob_dataset.to_pandas_dataframe()

thread 'tokio-runtime-worker' panicked at 'send should succeed: SendError { .. }', /src/dataprep/Core/rust_lex/rslex-http-stream/src/http_client/hyper_client/execution.rs:85:37
note: run with `RUST_BACKTRACE=1` environment variable to display a backtrace


In [2]:
##########################

week_ending = '2023-10-21'

##########################

date = datetime.strptime(week_ending, "%Y-%m-%d")
start_date = date - timedelta(days = 6)
end_date = date

In [3]:
tcp_df = tcp_df_dev.copy()

In [4]:
###TCP Data Manipulation
#Filter: Solar Jobs only 
tcp_df = tcp_df.loc[tcp_df['Job_Code'].str.startswith('710')]
#Filter: Only Keeping Labor Hourly
tcp_df = tcp_df.loc[tcp_df['Task'].isin(['LH','LHA'])]
#Filter: Only keeping Moss Labor Hourly - excluding subs
tcp_df = tcp_df.loc[tcp_df['Department']=='710']
#Creating a total hours column
tcp_df['total_hours'] = tcp_df['Regular_Hours'] + tcp_df['Overtime_Hours']
#Renaming Relevant Columns
tcp_df = tcp_df.rename({'Employee_Number':'EEID','Job_Code':'Job','Activity':'Sub_Job','total_hours':'Hours'},axis='columns')
tcp_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
tcp_df_comp = tcp_df.loc[:,tcp_cols_keep]

In [5]:
wd_pr_df = wd_pr_df_dev.copy()

In [ ]:
#'timeEntryCode.id1' .notna()
#'historicalStatus' != 94e8c24204d1406897f2758bd5e306d1
#'isDeleted' == False

In [6]:
#Workday Data Manipulation
#DUPLICATE FILTER - from WQL Query for Reported time Blocks for a Worker
wql_filter = (wd_pr_df['timeEntryCode.id1'].notna())&(wd_pr_df['historicalStatus']!='94e8c24204d1406897f2758bd5e306d1')&(wd_pr_df[''])

#Drop Null values if the employee does not have a job/sub job assignment
wd_pr_df = wd_pr_df.loc[~wd_pr_df['billableProject.descriptor'].isna()]
#Filter: Solar Jobs Only
wd_pr_df = wd_pr_df.loc[wd_pr_df['billableProject.descriptor'].str.startswith('710')]
#Extracting Employee ID (EEID) from 'Workers' column: This column also designates employees as Terminated so the substring '(Terminated)' is being removed for a clean creation of EEID from Workers
wd_pr_df['Workers'] = wd_pr_df['Workers'].str.replace('(Terminated)' , '')
wd_pr_df['EEID'] = wd_pr_df['Workers'].str.split('(').str[1]
wd_pr_df['EEID'] = wd_pr_df['EEID'].str.split(')').str[0]
#This isolates the 2 digit sub job, also accomodates the sub jobs that have a letter in them like Claims (C) and Warranty (W)
wd_pr_df['billableProject.descriptor'] = wd_pr_df['billableProject.descriptor'].str.replace('C' , '.C')
wd_pr_df['billableProject.descriptor'] = wd_pr_df['billableProject.descriptor'].str.replace('W' , '.W')
wd_pr_df['sub_job'] = wd_pr_df['billableProject.descriptor'].str.split('.').str[1]
wd_pr_df['sub_job'] = wd_pr_df['sub_job'].str.split(' ').str[0]
#Leaves behind Job ONLY
wd_pr_df['job'] = wd_pr_df.loc[:,'billableProject.descriptor'].str[0:7]
#Renaming Relevant Columns
wd_pr_df = wd_pr_df.rename({'calculatedDate':'Date','job':'Job','sub_job':'Sub_Job','amount':'Hours'},axis='columns')
wd_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
wd_pr_df_comp = wd_pr_df.loc[:,wd_cols_keep]
#excluding these employees from Workday because they will not be present in the TCP data
wd_eeid_exclude = ['6423','6525','9633','9647','9686','11780','13401','13757','14452','14523','14587','14594','15312','15358','15681','15912']
wd_pr_df_comp = wd_pr_df_comp.loc[~wd_pr_df_comp['EEID'].isin(wd_eeid_exclude)]

In [7]:
cmic_pr_df = cmic_pr_df_dev.copy()

In [8]:
#CMiC Data Manipulation
#Re-casting this column as a Date data type
cmic_pr_df["Date"] = pd.to_datetime(cmic_pr_df["Date"])
#Filter: Only Keeping Labor Hourly
cmic_pr_df = cmic_pr_df.loc[cmic_pr_df['Task'].isin(['LH','LHA'])]
#Renaming Relevant Columns
cmic_pr_df = cmic_pr_df.rename({'Employee Number':'EEID','Job Code':'Job','Sub Job Code':'Sub_Job','Regular Hours':'Hours'},axis='columns')
cmic_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
cmic_pr_df_comp = cmic_pr_df.loc[:,cmic_cols_keep]

In [9]:
#Date Filters
tcp_filter = (tcp_df_comp['Date']>=start_date)&(tcp_df_comp['Date']<=end_date)
cmic_filter = (cmic_pr_df_comp['Date']>=start_date)&(cmic_pr_df_comp['Date']<=end_date)
wd_filter = (wd_pr_df_comp['Date']>=start_date)&(wd_pr_df_comp['Date']<=end_date)

#### Grouping the Data by Different Levels

In [10]:
#Grouping by Job
tcp_grouped_job = tcp_df_comp.loc[tcp_filter].groupby(['Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_job = tcp_grouped_job.reset_index()
cmic_grouped_job = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_job = cmic_grouped_job.reset_index()
wd_grouped_job = wd_pr_df_comp.loc[wd_filter].groupby(['Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_job = wd_grouped_job.reset_index()

In [11]:
#Grouping by Job, Sub Job
tcp_grouped_subjob = tcp_df_comp.loc[tcp_filter].groupby(['Job','Sub_Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_subjob = tcp_grouped_subjob.reset_index()
cmic_grouped_subjob = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job','Sub_Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_subjob = cmic_grouped_subjob.reset_index()
wd_grouped_subjob = wd_pr_df_comp.loc[wd_filter].groupby(['Job','Sub_Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_subjob = wd_grouped_subjob.reset_index()

In [12]:
#Grouping by Job, Sub Job, EEID
tcp_grouped_ee = tcp_df_comp.loc[tcp_filter].groupby(['EEID']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_ee = tcp_grouped_ee.reset_index()
cmic_grouped_ee = cmic_pr_df_comp.loc[cmic_filter].groupby(['EEID']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_ee = cmic_grouped_ee.reset_index()
wd_grouped_ee = wd_pr_df_comp.loc[wd_filter].groupby(['EEID']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_ee = wd_grouped_ee.reset_index()

#### Unique Column Creation for the Joins

In [13]:
#Unique Columns to Join On for Sub Job Comparison
tcp_grouped_subjob['unique'] = tcp_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
cmic_grouped_subjob['unique'] = cmic_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
wd_grouped_subjob['unique'] = wd_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)

#### TCP vs. Workday Comparison

In [14]:
#TCP v. WD Job
tcp_v_wd_job = tcp_grouped_job.merge(wd_grouped_job , how = 'left' , on = 'Job' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_job['TotalHours_WD'] = tcp_v_wd_job['TotalHours_WD'].fillna(0)
tcp_v_wd_job['TotalHours_TCP'] = tcp_v_wd_job['TotalHours_TCP'].fillna(0)
tcp_v_wd_job['Hours_Variance (TCP - WD)'] = tcp_v_wd_job['TotalHours_TCP'] - tcp_v_wd_job['TotalHours_WD']
tcp_v_wd_job = tcp_v_wd_job.sort_values(['Job'  , 'Hours_Variance (TCP - WD)'] , ascending=[False,False])

In [15]:
#TCP v. WD Job, Sub Job
tcp_v_wd_subjob = tcp_grouped_subjob.merge(wd_grouped_subjob , how = 'left' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_wd_subjob['TotalHours_WD'] = tcp_v_wd_subjob['TotalHours_WD'].fillna(0)
tcp_v_wd_subjob['TotalHours_TCP'] = tcp_v_wd_subjob['TotalHours_TCP'].fillna(0)
tcp_v_wd_subjob['Hours_Variance (TCP - WD)'] = tcp_v_wd_subjob['TotalHours_TCP'] - tcp_v_wd_subjob['TotalHours_WD']
tcp_v_wd_subjob = tcp_v_wd_subjob.sort_values(['Job_TCP' , 'Sub_Job_TCP' , 'Hours_Variance (TCP - WD)'] , ascending=[False,True,False])

In [16]:
#TCP v. WD Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'left' , on = 'EEID' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_ee['TotalHours_WD'] = tcp_v_wd_ee['TotalHours_WD'].fillna(0)
tcp_v_wd_ee['TotalHours_TCP'] = tcp_v_wd_ee['TotalHours_TCP'].fillna(0)
tcp_v_wd_ee['Hours_Variance (TCP - WD)'] = tcp_v_wd_ee['TotalHours_TCP'] - tcp_v_wd_ee['TotalHours_WD']
tcp_v_wd_ee = tcp_v_wd_ee.sort_values('Hours_Variance (TCP - WD)' , ascending=False, key=abs)
#Removing Variances that are 0 - excluding EEIDs that match
tcp_v_wd_ee_xl = tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0.0]

#### TCP vs. CMiC Comparison

In [17]:
#TCP v CMiC Job
tcp_v_cmic_job = tcp_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_job['TotalHours_CMiC'] = tcp_v_cmic_job['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_job['TotalHours_TCP'] = tcp_v_cmic_job['TotalHours_TCP'].fillna(0)
tcp_v_cmic_job['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_job['TotalHours_TCP'] - tcp_v_cmic_job['TotalHours_CMiC']
tcp_v_cmic_job = tcp_v_cmic_job.sort_values(['Job' , 'Hours_Variance (TCP - CMiC)'], ascending=[False, False])

In [18]:
#TCP v CMiC Job, Sub Job
tcp_v_cmic_subjob = tcp_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_cmic_subjob['TotalHours_CMiC'] = tcp_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_subjob['TotalHours_TCP'] = tcp_v_cmic_subjob['TotalHours_TCP'].fillna(0)
tcp_v_cmic_subjob['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_subjob['TotalHours_TCP'] - tcp_v_cmic_subjob['TotalHours_CMiC']
tcp_v_cmic_subjob = tcp_v_cmic_subjob.sort_values(['Job' , 'Sub_Job', 'Hours_Variance (TCP - CMiC)'], ascending=[False, True, False])

In [19]:
#TCP v CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_ee['TotalHours_CMiC'] = tcp_v_cmic_ee['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_ee['TotalHours_TCP'] = tcp_v_cmic_ee['TotalHours_TCP'].fillna(0)
tcp_v_cmic_ee['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_ee['TotalHours_TCP'] - tcp_v_cmic_ee['TotalHours_CMiC']
tcp_v_cmic_ee = tcp_v_cmic_ee.sort_values('Hours_Variance (TCP - CMiC)', ascending=False, key=abs)
#Filter down to relevant columns only

#### Workday vs. CMiC Comparison

In [20]:
#WD v. CMiC EEID
wd_v_cmic_job = wd_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_job['TotalHours_CMiC'] = wd_v_cmic_job['TotalHours_CMiC'].fillna(0)
wd_v_cmic_job['TotalHours_WD'] = wd_v_cmic_job['TotalHours_WD'].fillna(0)
wd_v_cmic_job['Hours_Variance (WD - CMiC)'] = wd_v_cmic_job['TotalHours_WD'] - wd_v_cmic_job['TotalHours_CMiC']
wd_v_cmic_job = wd_v_cmic_job.sort_values(['Job' , 'Hours_Variance (WD - CMiC)'], ascending=[False,False])

In [21]:
#WD v. CMiC Job, Sub Job
wd_v_cmic_subjob = wd_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
wd_v_cmic_subjob['TotalHours_CMiC'] = wd_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
wd_v_cmic_subjob['TotalHours_WD'] = wd_v_cmic_subjob['TotalHours_WD'].fillna(0)
wd_v_cmic_subjob['Hours_Variance (WD - CMiC)'] = wd_v_cmic_subjob['TotalHours_WD'] - wd_v_cmic_subjob['TotalHours_CMiC']
wd_v_cmic_subjob = wd_v_cmic_subjob.sort_values(['Job_WD' , 'Sub_Job_WD' , 'Hours_Variance (WD - CMiC)'], ascending=[False,True,False])

In [22]:
#WD v. CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_ee['TotalHours_CMiC'] = wd_v_cmic_ee['TotalHours_CMiC'].fillna(0)
wd_v_cmic_ee['TotalHours_WD'] = wd_v_cmic_ee['TotalHours_WD'].fillna(0)
wd_v_cmic_ee['Hours_Variance (WD - CMiC)'] = wd_v_cmic_ee['TotalHours_WD'] - wd_v_cmic_ee['TotalHours_CMiC']
wd_v_cmic_ee = wd_v_cmic_ee.sort_values('Hours_Variance (WD - CMiC)', ascending=False , key=abs)
#Filter down to relevant columns only

In [23]:
file_path = os.getcwd() + '/excel_output'
file_path

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jbeachy2/code/Users/JBeachy/Project_DataDomain/Payroll/git_LaborHours_Compare/excel_output'

In [24]:
#File Path in AMLS
file_name = f'Hours_Comparison_WeekEnding_{week_ending}'
#Writing to AMLS File Directory
with pd.ExcelWriter(f'{file_path}/{file_name}.xlsx') as writer:
    tcp_v_wd_job.to_excel(writer , sheet_name = 'TCPvWD_Hours_byJob' , index = False)
    tcp_v_wd_subjob.to_excel(writer , sheet_name = 'TCPvWD_Hours_bySubJob' , index = False)
    tcp_v_wd_ee_xl.to_excel(writer , sheet_name = 'TCPvWD_Hours_byEE' , index = False)
    tcp_v_cmic_job.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byJob' , index = False)
    tcp_v_cmic_subjob.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_bySubJob' , index = False)
    tcp_v_cmic_ee.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byEE' , index = False)
    wd_v_cmic_job.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byJob' , index = False)
    wd_v_cmic_subjob.to_excel(writer , sheet_name = 'WDvCMiC_Hours_bySubJob' , index = False)
    wd_v_cmic_ee.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byEE' , index = False)

In [25]:
#Blob Storage Info
storage_account_key = 'u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA=='
storage_account_name = 'mossdatalakesource'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=mossdatalakesource;AccountKey=u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA==;EndpointSuffix=core.windows.net'
container_name = 'cmic'
#Defininng Blob Storage Uplpoad#
def uploadtoblobstorage(file_path,file_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container = container_name , blob = file_name)

    with open(file_path , 'rb') as data:
        blob_client.upload_blob(data)
    print(f'Uploaded {file_name}.')

In [26]:
file_name_exp = f'{file_name}.xlsx'
uploadtoblobstorage(f'{file_path}/{file_name}.xlsx', file_name_exp)

Uploaded Hours_Comparison_WeekEnding_2023-10-21.xlsx.


#### Run Statistics 

In [27]:
#### TCP vs. Workday Stats
today = date.today()
today_timechange = today - timedelta(hours=4)
today_for_text = today_timechange.strftime("%m-%d-%Y %H:%M:%S")
print("TCP vs. Workday")
print("Count of EEIDs that are showing Zero Hours Worked in Workday: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_WD'] == 0]))
print("Count of EEIDs that are showing Zero Hours Worked in TCP: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_TCP'] == 0]))
print("Count of EEIDs with a variance less than 10 hours(ABS), not including 0: ", len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']< 10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] >-10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0)]))
print("Count of EEIDs with no variance in Hours Worked: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] == 0]))
print("Count of EEIDs with variance greater than or equal to 10(ABS): ",len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']>=10)|(tcp_v_wd_ee['Hours_Variance (TCP - WD)']<=-10)]))
print(f"Ran on {today_for_text} for Week Ending {week_ending}")

TCP vs. Workday
Count of EEIDs that are showing Zero Hours Worked in Workday:  55
Count of EEIDs that are showing Zero Hours Worked in TCP:  19
Count of EEIDs with a variance less than 10 hours(ABS), not including 0:  721
Count of EEIDs with no variance in Hours Worked:  1581
Count of EEIDs with variance greater than or equal to 10(ABS):  353
Ran on 10-23-2023 15:53:47 for Week Ending 2023-10-21


In [3]:
investigate_df = wd_pr_df_dev.loc[(wd_pr_df_dev['Workers'].str.contains('10931'))&(wd_pr_df_dev['billableTransaction.descriptor'].str.contains('10/19/2023'))]

In [6]:
#EmployeeID, Employee_FullName, Hours



#cf_TT_TimeBlockNotSubmitted , isDeleted , originalReportedQuantity , status.descriptor , status.descriptor1
cols = ['EmployeeID', 'Employee_FullName','amount','cf_TT_TimeBlockNotSubmitted','originalReportedQuantity','status.descriptor','status.descriptor1','timeEntryCode.id1','historicalStatus','isDeleted']
#nunique = investigate_df.nunique()
#cols_to_drop = nunique[nunique == 1].index
#investigate_df = investigate_df.drop(cols_to_drop, axis=1)
#investigate_df = investigate_df.dropna(axis=1)
investigate_df.loc[:,cols]#['Employee_BillTransaction_Descriptor'].nunique()

,EmployeeID,Employee_FullName,amount,cf_TT_TimeBlockNotSubmitted,originalReportedQuantity,status.descriptor,status.descriptor1,timeEntryCode.id1,historicalStatus,isDeleted
95608,10931,Emily Easterling,6.5,False,0.0,Never Submitted,Never Submitted,77c9ae7eb96b1001b5ebf29ffdea0000,None,True
97513,10931,Emily Easterling,10.0,True,10.0,Not Submitted,Not Submitted,77c9ae7eb96b1001b5ebf29ffdea0000,None,False
98502,10931,Emily Easterling,3.5,False,0.0,Never Submitted,Never Submitted,None,None,True


In [30]:
wd_pr_df_comp.loc[(wd_pr_df_comp['EEID']=='10931')&wd_filter].sort_values('Date')

,EEID,Date,Job,Sub_Job,Hours
100287,10931,2023-10-16,7102230,50,10.00
100728,10931,2023-10-16,7102230,50,10.00
100288,10931,2023-10-17,7102230,50,6.75
100729,10931,2023-10-17,7102230,50,6.75
100289,10931,2023-10-19,7102230,50,6.50
100730,10931,2023-10-19,7102230,50,10.00
101214,10931,2023-10-19,7102230,50,3.50
100290,10931,2023-10-20,7102230,50,5.25
100731,10931,2023-10-20,7102230,50,5.25


In [190]:
wd_pr_df_dev.loc[wd_pr_df_dev['Workers'].str.contains('4082'),['Workers','FrequencySalary',]]

,workdayID,EmployeeID,Employee_FullName,Workers,Employee_BillTransaction_Descriptor,billableTransaction.descriptor,Employee_BillTransactionDate,amount,dayOfTheWeek,FrequencySalary,...,timeType1.id,totalHoursTimeTracking,transactionDate2,transactionSource.descriptor,transactionSource.id,unit.descriptor,unit.id,unit1,unitOfTimeIsHours,unroundedDuration
43021,7c8f1553b88990011947d733e8c20000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/02/2023,10/02/2023,10.0,Monday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-02,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
43022,7c8f1553b88990011947d86850e10000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/03/2023,10/03/2023,10.0,Tuesday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-03,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
43023,7c8f1553b88990011947d9029b6b0000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/04/2023,10/04/2023,10.0,Wednesday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-04,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
43024,7c8f1553b88990011947d99d041c0000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/05/2023,10/05/2023,10.0,Thursday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-05,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
50602,7c8f1553b8899002172e24b276f40000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/09/2023,10/09/2023,10.0,Monday,Weekly Unions,...,98e25664125510010c8407187b230000,10.0,2023-10-09,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
50603,7c8f1553b8899002172e254d60670000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/10/2023,10/10/2023,10.0,Tuesday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-10,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
50604,7c8f1553b8899002172e25e719c70000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/11/2023,10/11/2023,10.0,Wednesday,Weekly Unions,...,98e25664125510010c8407187b230000,10.0,2023-10-11,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
50605,7c8f1553b8899002172e268098770000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/12/2023,10/12/2023,10.0,Thursday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-12,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
51896,dcd88da845b59002168f57a38a360000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/09/2023,10/09/2023,10.0,Monday,Weekly Unions,...,98e25664125510010c8407187b230000,10.0,2023-10-09,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
51897,dcd88da845b59002168f583e79ab0000,4082,Aisoli Vitale,Aisoli Vitale (4082),10 Hours,10 Hours on 10/10/2023,10/10/2023,10.0,Tuesday,Weekly Unions,...,88572bc8f2d81000d2eb4ff7cc130000,10.0,2023-10-10,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0


In [196]:
#Are these employees all Hawaii Union?
double_eeid = tcp_v_wd_ee_xl.loc[tcp_v_wd_ee_xl['Hours_Variance (TCP - WD)'] < -11.5 , 'EEID'].unique()

In [203]:

wd_pr_df.loc[wd_pr_df['EEID'].isin(double_eeid) , 'FrequencySalary'].unique()

array(['Weekly Unions'], dtype=object)

In [27]:
tcp_v_wd_ee_xl #.loc[tcp_v_wd_ee_xl['Hours_Variance (TCP - WD)'] < -11.5]

,EEID,TotalHours_TCP,TotalHours_WD,Hours_Variance (TCP - WD)
2466,8185,40.00,80.0,-40.00
919,15339,40.00,80.0,-40.00
2048,4082,40.00,80.0,-40.00
2046,3943,50.00,90.0,-40.00
2045,3912,60.00,100.0,-40.00
2044,3859,50.00,90.0,-40.00
2043,3817,40.00,80.0,-40.00
2042,3811,40.00,80.0,-40.00
584,14422,40.00,80.0,-40.00
2251,6652,50.00,90.0,-40.00


In [28]:
#Check this after truncate and reload
wd_pr_df.loc[wd_pr_df['EEID']=='15990']

,workdayID,EmployeeID,Employee_FullName,Workers,Employee_BillTransaction_Descriptor,billableTransaction.descriptor,Employee_BillTransactionDate,Hours,dayOfTheWeek,FrequencySalary,...,transactionSource.descriptor,transactionSource.id,unit.descriptor,unit.id,unit1,unitOfTimeIsHours,unroundedDuration,EEID,Sub_Job,Job
29794,e70ba2f2f4839000b46c3ac88fb80000,15990,Kenneth Simmions,Kenneth Simmions (15990),8 Hours,8 Hours on 09/25/2023,09/25/2023,8.0,Monday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
29795,e70ba2f2f4839000b46c3b6443430000,15990,Kenneth Simmions,Kenneth Simmions (15990),5 Hours,5 Hours on 09/26/2023,09/26/2023,5.0,Tuesday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
29796,e70ba2f2f4839000b46c3c989fec0000,15990,Kenneth Simmions,Kenneth Simmions (15990),8 Hours,8 Hours on 09/27/2023,09/27/2023,8.0,Wednesday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
29797,e70ba2f2f4839000b46c3d32b4510000,15990,Kenneth Simmions,Kenneth Simmions (15990),8 Hours,8 Hours on 09/28/2023,09/28/2023,8.0,Thursday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
29798,e70ba2f2f4839000b46c3dcc82430000,15990,Kenneth Simmions,Kenneth Simmions (15990),2 Hours,2 Hours on 09/29/2023,09/29/2023,2.0,Friday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
52565,7c8f1553b8899000bc03a0e39ef70000,15990,Kenneth Simmions,Kenneth Simmions (15990),8 Hours,8 Hours on 10/03/2023,10/03/2023,8.0,Tuesday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
52566,7c8f1553b8899000bc03a17d8c5c0000,15990,Kenneth Simmions,Kenneth Simmions (15990),8 Hours,8 Hours on 10/04/2023,10/04/2023,8.0,Wednesday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
52567,7c8f1553b8899000bc03a2b117ba0000,15990,Kenneth Simmions,Kenneth Simmions (15990),5 Hours,5 Hours on 10/05/2023,10/05/2023,5.0,Thursday,None,...,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0,15990,15,7102235
